In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import ndimage as ndi

from skimage.morphology import watershed
from skimage.feature import peak_local_max
from PIL import Image
from skimage.morphology import remove_small_objects


from skimage.morphology import skeletonize, thin
from skimage.util import invert
from skimage import measure
from skimage.morphology import erosion, dilation, opening, closing, white_tophat
from skimage.morphology import black_tophat, skeletonize, convex_hull_image
from skimage.morphology import disk
from skimage import feature
from skimage.filters import roberts, sobel, scharr, prewitt
import seaborn as sns
sns.reset_orig()

from skimage import restoration

from skimage import exposure
from skimage.filters import gaussian

from scipy import ndimage

from time import time

from skimage.color import rgb2gray
from scipy import ndimage as ndi

from skimage.filters.rank import entropy
import scipy


import networkx as nx
from scipy import interpolate

from skimage.segmentation import random_walker
from skimage.data import binary_blobs
from skimage.exposure import rescale_intensity
import glob

from skimage.morphology import disk
from skimage import io

from skimage.color import rgb2hsv
import cv2

from skimage.external import tifffile

import pandas as pd

from sklearn.cluster import KMeans

from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import DBSCAN

import threading

/Users/spencerlab/anaconda3/lib/python3.7/site-packages/matplotlib/__init__.py:855: MatplotlibDeprecationWarning: 
examples.directory is deprecated; in the future, examples will be found relative to the 'datapath' directory.
  "found relative to the 'datapath' directory.".format(key))
/Users/spencerlab/anaconda3/lib/python3.7/site-packages/matplotlib/__init__.py:846: MatplotlibDeprecationWarning: 
The text.latex.unicode rcparam was deprecated in Matplotlib 2.2 and will be removed in 3.1.
  "2.2", name=key, obj_type="rcparam", addendum=addendum)


In [2]:
def color_leave(arr, h_min, h_max, s_min, s_max, v_min, v_max):
    arr2= arr*0
    for i in range(len(arr)):
        arr2[i] =  cv2.cvtColor(arr[i],cv2.COLOR_RGB2HSV)
        
    
    if h_max<h_min:
        h_max_2 = h_max.copy()
        h_max = h_min
        h_min = h_max_2
    
    
    m = (arr2[:, :,:,0]>=h_min)*(arr2[:, :,:,0]<=h_max)*(arr2[:,:,:,1]>=s_min)*(arr2[:,:,:,1]<=s_max)*(arr2[:,:,:,2]>=v_min)*(arr2[:,:,:,2]<=v_max)
    
    return m
    

In [3]:
def get_occurances(markers, points):
    
    
    pt_labels = markers[points[0], points[1], points[2]]

    label_occurances = (np.bincount(pt_labels, weights=np.ones((points.shape[1]))) )[1:]
    
    
    return label_occurances

In [4]:
def get_centroids(arr):
    
    arr= arr!=0
    markers, nummarkers = ndi.label(arr) 
    
    points = np.where(arr)
    points = np.asarray(points)
    
    pt_labels = markers[points[0], points[1], points[2]]
    
    Z_mass = (np.bincount(pt_labels, weights=points[0]))[1:]

    Y_mass = (np.bincount(pt_labels, weights=points[1]))[1:]

    X_mass = (np.bincount(pt_labels, weights=points[2]))[1:]

    label_occurances = (np.bincount(pt_labels, weights=np.ones((points.shape[1]))) )[1:]
    
    
    Z_centroid = Z_mass/label_occurances
    Y_centroid = Y_mass/label_occurances
    X_centroid = X_mass/label_occurances
    
    return Z_centroid, Y_centroid, X_centroid, label_occurances

In [5]:
c

In [6]:
#stack = io.imread('/Users/spencerlab/Desktop/final_files/Day_2/#1/715_2d_RGB_copy_p.tif')
#stack = io.imread('/Users/spencerlab/Desktop/final_files/Day_2/#2/721_2d_RGB_p.tif')
stack = io.imread('/Users/spencerlab/Desktop/final_files/Day_3/#3/722_3d_RGB_p.tif')

#stack = io.imread('/Users/spencerlab/Desktop/final_files/Day_5/Stack_day_5.tif')

In [7]:
#PARAMETERS
h_min = 52
s_min = 141
v_min = 145

h_max = 116
s_max = 255
v_max = 255


MINIMUM_SIZE = 12


In [8]:
print("Precompute")
filtered_stack = color_leave(stack, h_min, h_max, s_min, s_max, v_min, v_max)

#PRECOMPUTE AND UNMODULARIZE
filtered_stack= filtered_stack!=0
markers, nummarkers = ndi.label(filtered_stack) 
    
points = np.where(filtered_stack)
points = np.asarray(points)

Precompute


In [9]:
print("Process")
occ = get_occurances(markers, points)
sizemap_stack =  pair_labels_W_vals(occ, markers, nummarkers, points)
Z_centroid, Y_centroid, X_centroid, label_occurances = get_centroids(sizemap_stack>=MINIMUM_SIZE)

Process


In [10]:
print("Positions")
data = np.zeros((4, Z_centroid.shape[0]), dtype= int)
data[0] = Z_centroid
data[1] = Y_centroid
data[2] = X_centroid
data[3] = label_occurances
data = np.transpose(data)

df = pd.DataFrame(data=data, columns=['Z', 'Y', 'X', 'Size-voxels'])

Positions


In [11]:
df

,Z,Y,X,Size-voxels
0,14,466,3021,19
1,17,180,2992,12
2,18,153,2219,63
3,18,212,2233,74
4,21,39,2155,98
...,...,...,...,...
70,61,28,2277,42
71,70,83,2182,31
72,76,167,2442,44
73,105,188,2250,42


In [12]:
df.to_csv('/Users/spencerlab/Downloads/Day_3-3.csv')